In [1]:
# imports:
import time;
import urllib.request
import csv
import os
import itertools
import logging
import json
import numpy as np
import gensim
from bs4 import BeautifulSoup
from urllib.parse import urlencode
from gensim.utils import smart_open, simple_preprocess
from gensim.corpora.wikicorpus import _extract_pages, filter_wiki
from gensim.parsing.preprocessing import STOPWORDS
from xml.etree.cElementTree import iterparse
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import neighbors
from sklearn import svm

logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO

id2word_wiki=gensim.corpora.Dictionary()

D:\Work\Anaconda\envs\py35\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# List Categories:
#category_list = ["Mathematics","Technology","Music"]
category_list=["Arts", "Geography", "Health", "History", "Literature", "Mathematics", "Music", "Nature", "Religion", "Technology"]
#category_list = ["1977 introductions", "Language" ,"Arts", "Asia", "Asthma", "Automobiles", "BRICS nation", "Belief", "Climate", "Crime", "Culture", "Dance", "Deserts", "Dolls", "Earth", "Engines", "Fishing", "Folklore", "Games", "Geography", "Glass", "Government agencies", "Health", "History", "Humans", "Hygiene", "India", "Industry", "Internet", "Law", "Life", "Literature", "Mathematics", "Matter", "Millionaires", "Music", "Nature", "Nothing", "Parties", "Peace", "Politics", "Religion", "Sexology", "Society", "Songs", "Space", "Technology", "Television", "Transport", "Water sports"]
test_folder =  "./test/"
root_folder = "./Simplex/"
list.sort(category_list)
folder_name=''.join([x[0]+x[1]+x[2] for x in category_list])
root_folder='./'+folder_name+'/'

print("Checking folder : "+folder_name)
if not os.path.exists(root_folder):
    os.mkdir(root_folder)

    

wiki_bow_path = root_folder+'wiki_bow.mm'
wiki_dict_path = root_folder+'wiki_dict.dict'

Checking folder : ArtGeoHeaHisLitMatMusNatRelTec


In [3]:
# Download Page Ids:
#https://petscan.wmflabs.org/?language=en&project=wikipedia&depth=1&format=csv&categories=mathematics&doit=Do it!
print ("Scanning CSV")
for cat in category_list:
    if not os.path.exists(root_folder+cat+".csv"):
        url="https://petscan.wmflabs.org/?language=en&project=wikipedia&depth=1&format=csv&doit=Do%20it!&categories="+cat
        urllib.request.urlretrieve(url, root_folder+cat+".csv")
        print("Downloading ",cat+".csv")
    

Scanning CSV


In [4]:
# CSV to XML Download data:

def get_data(ids,output_file):
    url="https://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&format=xml&pageids="+ids
    req = urllib.request.urlopen(url)
    if req.getcode() == 200:
        soup = BeautifulSoup(req.read(), 'html.parser')
        s = soup.find_all('page')
        for si in s:
            output_file.write(str(si))

def batch_train(file):
    output_file = open(file.replace(".csv",".xml"), 'a', encoding="utf8")
    output_file.write("<pages>")
                    
    csvReader = csv.reader(open(file,'r'))
    totalRecords = sum(1 for row in csv.reader(open(file,'r',encoding="UTF-8")) )
    print (file.replace(".csv",".xml"),totalRecords)
    start = 0
    end = start + 50
   
    while (start <= totalRecords):
        pageIds = ""
        for row in itertools.islice(csv.reader(open(file,'r',encoding="UTF-8")),start,end):
            pageIds = pageIds + row[2] + "|"
        
        get_data(pageIds,output_file)
        start = end + 1
        end = start + 50
        if end> totalRecords:
            end=totalRecords
            
    output_file.write("</pages>")
    

for path, subdirs, files in os.walk(root_folder):
    for file in files:
        if file.endswith('.csv'):
            if not os.path.exists(root_folder+file.replace(".csv",".xml")):
                print("Downloading ",root_folder+file.replace(".csv",".xml"))
                batch_train(path + file)

print ("Done downloading")

Done downloading


In [5]:
# Train Model
def head(stream, n=10):
    """Convenience fnc: return the first `n` elements of the stream, as plain list."""
    return list(itertools.islice(stream, n))

def my_extract_pages(f):
    elems = (elem for _, elem in iterparse(f, events=("end",)))
    page_tag = "rev"
    for elem in elems:
        if elem.tag == page_tag and elem.text != None:
            text = elem.text
            yield text
            elem.clear()

def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

def iter_wiki(dump_file):
    for text in my_extract_pages(smart_open(dump_file)):
        text = filter_wiki(text)
        tokens = tokenize(text)
        if len(tokens) < 50:
            continue  # ignore short articles and various meta-articles
        yield tokens
        

In [6]:
print("Training Model")
class WikiCorpus(object):
    def __init__(self, dump_file, dictionary, clip_docs=None):
        """
        Parse the first `clip_docs` Wikipedia documents from file `dump_file`.
        Yield each document in turn, as a list of tokens (unicode strings).
        
        """
        self.dump_file = dump_file
        self.dictionary = dictionary
        self.clip_docs = clip_docs
    
    def __iter__(self):
        for tokens in itertools.islice(iter_wiki(self.dump_file), self.clip_docs):
            yield self.dictionary.doc2bow(tokens)
    
    def __len__(self):
        return self.clip_docs


if not (os.path.exists(wiki_bow_path) and  os.path.exists(wiki_dict_path)):
    for path, subdirs, files in os.walk(root_folder):
        del subdirs[:]
        for file in files:
            if file.endswith('.xml'):
                doc_path = path + file
                stream = iter_wiki(doc_path)
                doc_stream = (tokens for tokens in iter_wiki(doc_path))
                id2word_wiki.merge_with(gensim.corpora.Dictionary(doc_stream))

    id2word_wiki.filter_extremes(no_below=10, no_above=0.1)
    
    # create a stream of bag-of-words vectors
    wiki_corpus = WikiCorpus(doc_path, id2word_wiki)
    
    id2word_wiki.save(wiki_dict_path) 
    gensim.corpora.MmCorpus.serialize(wiki_bow_path, wiki_corpus)
print(len(category_list))
id2word_wiki = gensim.corpora.Dictionary.load(wiki_dict_path)
mm_corpus = gensim.corpora.MmCorpus(wiki_bow_path)
clipped_corpus = gensim.utils.ClippedCorpus(mm_corpus, 4000) 


Training Model
10


INFO : loading Dictionary object from ./ArtGeoHeaHisLitMatMusNatRelTec/wiki_dict.dict
INFO : loaded ./ArtGeoHeaHisLitMatMusNatRelTec/wiki_dict.dict
INFO : loaded corpus index from ./ArtGeoHeaHisLitMatMusNatRelTec/wiki_bow.mm.index
INFO : initializing corpus reader from ./ArtGeoHeaHisLitMatMusNatRelTec/wiki_bow.mm
INFO : accepted corpus with 1968 documents, 37814 features, 419342 non-zero entries


In [7]:
lda_model = gensim.models.LdaModel(clipped_corpus, num_topics=len(category_list), id2word=id2word_wiki, passes=10, alpha='asymmetric')
print("done")

INFO : using asymmetric alpha [0.20349776650601445, 0.15460680268266819, 0.12465746387131386, 0.10442834231175278, 0.089848016434062955, 0.078840302634700543, 0.070235422324486096, 0.063324036444942805, 0.05765099744942407, 0.052910849340634232]
INFO : using symmetric eta at 2.6445231924683978e-05
INFO : using serial LDA version on this node
INFO : running online LDA training, 10 topics, 10 passes over the supplied corpus of 1968 documents, updating model once every 1968 documents, evaluating perplexity every 1968 documents, iterating 50x with a convergence threshold of 0.001000
INFO : -11.623 per-word bound, 3153.7 perplexity estimate based on a held-out corpus of 1968 documents with 804398 words
INFO : PROGRESS: pass 0, at document #1968/1968
INFO : topic #9 (0.053): 0.005*"mobile" + 0.003*"digital" + 0.003*"engineering" + 0.003*"standards" + 0.003*"user" + 0.002*"users" + 0.002*"internet" + 0.002*"million" + 0.002*"metadata" + 0.002*"web"
INFO : topic #8 (0.058): 0.004*"digital" + 0

done


In [8]:
tfidf_model = gensim.models.TfidfModel(mm_corpus, id2word=id2word_wiki)
lsi_model = lsi_model = gensim.models.LsiModel(tfidf_model[mm_corpus], id2word=id2word_wiki, num_topics=len(category_list))
print("done")

INFO : collecting document frequencies
INFO : PROGRESS: processing document #0
INFO : calculating IDF weights for 1968 documents and 37813 features (419342 matrix non-zeros)
INFO : using serial LSI version on this node
INFO : updating model with new documents
INFO : preparing a new chunk of documents
INFO : using 100 extra samples and 2 power iterations
INFO : 1st phase: constructing (37814, 110) action matrix
INFO : orthonormalizing (37814, 110) action matrix
INFO : 2nd phase: running dense svd on (110, 1968) matrix
INFO : computing the final decomposition
INFO : keeping 10 factors (discarding 74.579% of energy spectrum)
INFO : processed documents up to #1968
INFO : topic #0(5.167): -0.289*"mobile" + -0.121*"engineering" + -0.116*"software" + -0.106*"digital" + -0.105*"user" + -0.096*"users" + -0.095*"internet" + -0.087*"web" + -0.085*"learning" + -0.084*"technologies"
INFO : topic #1(3.487): 0.576*"mobile" + -0.261*"engineering" + 0.143*"phone" + -0.115*"technological" + -0.110*"ener

done


In [9]:
def calculate_centroid(topic_docs):
    test_doc = [tokens for tokens in iter_wiki(topic_docs)]
    part = [lda_model[id2word_wiki.doc2bow(tokens)] for tokens in test_doc]
    
    topic_dic={}
    
    for i in range(len(category_list)):
        topic_dic[i]=0
        
    for doc in part:
        for p in doc:
            topic_dic[p[0]] += p[1]
    
    centroid = [(x, topic_dic[x]/len(part)) for x in range(len(category_list))]
    return centroid

print("Calcuating centroid")

centroids_dict={}
for path, subdirs, files in os.walk(root_folder):
    del subdirs[:]
    for file in files:
        if file.endswith('.xml'):
            doc_path = path + file
            centroid = calculate_centroid(doc_path)
            centroids_dict[file.replace(".xml","")]=centroid

print("Centroid calculation done")

Calcuating centroid
Centroid calculation done


In [10]:
km=None
def get_values(l):
    g=[0]*len(category_list)
    for i in l:
        g[i[0]]=i[1]
    return g

def calculate_kmeans(part):
    km = KMeans(n_clusters=len(category_list), init='k-means++', max_iter=500, n_init=1)
    km.fit(part)
    return km

print("Calculating KMeans ")
for path, subdirs, files in os.walk(root_folder):
    del subdirs[:]
    part=[]
    for file in files:
        if file.endswith('.xml'):
            doc_path = path + file
            test_doc = [tokens for tokens in iter_wiki(doc_path)]
            part+=[get_values(lda_model[id2word_wiki.doc2bow(tokens)]) for tokens in test_doc]
    km = calculate_kmeans(part)

Calculating KMeans 


In [40]:
y=[]
X=[]

for path, subdirs, files in os.walk(root_folder):
    del subdirs[:]
    for file in files:
        if file.endswith('.xml'):
            doc_path = path + file
            topic = file.replace(".xml","")
            
            test_doc = [tokens for tokens in iter_wiki(doc_path)]
            X+=[get_values(lda_model[id2word_wiki.doc2bow(tokens)]) for tokens in test_doc]
            y+=[category_list.index(topic)]*len(test_doc)

print("Feeding data to KNeighborsClassifier")
n_neighbors = 15
# we create an instance of Neighbours Classifier and fit the data.
clf = neighbors.KNeighborsClassifier(n_neighbors)
clf.fit(X, y)

Feeding data to KNeighborsClassifier


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=150, p=2,
           weights='uniform')

In [12]:
print("Feeding data to SVC")
C = 1.0  # SVM regularization parameter
svc = svm.SVC(kernel='linear', C=C).fit(X, y)
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=C).fit(X, y)
poly_svc = svm.SVC(kernel='poly', degree=3, C=C).fit(X, y)
lin_svc = svm.LinearSVC(C=C).fit(X, y)

Feeding data to SVC


In [13]:
def draw_graph(x_label,y,file,text_data,topic):
    x = np.arange(len(x_label))  # the x locations for the groups
    width = 0.1       # the width of the bars

    fig, ax = plt.subplots()
    fig.set_figheight(6)
    fig.set_figwidth(8)
    rects = ax.bar(x, y, width, color='blue')
    ax.set_ylabel('Probabilities')
    ax.set_xlabel('Categories')
    ax.set_title('Topic Distribution of: ' + topic)
    ax.title.set_position([.5, 1.2])
    ax.set_ylim([0,1])
    
    def autolabel(rects):
        """
        Attach a text label above each bar displaying its height
        """
        for rect in rects:
            height = rect.get_height()
            ax.text(rect.get_x() - rect.get_width()*2, 1.05 * height,round(height,2))
    

    autolabel(rects)
    plt.tight_layout(pad=6)
    plt.xticks(x,category_list,rotation=90)
    
    plt.savefig(file.replace(".xml","_")+str(len(category_list))+'.png')
    plt.close('all')

In [45]:
# Test LDA - Cosine
model_name="LDA-Cosine"
if not os.path.exists(root_folder+model_name):
    os.mkdir(root_folder+model_name)
    
def get_part(testFile):
    test_doc = [tokens for tokens in iter_wiki(testFile)]
    part = [lda_model[id2word_wiki.doc2bow(tokens)] for tokens in test_doc]
    return part

results=""
print("Testing results")
for path, subdirs, files in os.walk(test_folder):
    for file in files:
        if file.endswith('.xml'):
            doc_path = test_folder + file
            path=get_part(doc_path)
            print(file)
            results+=file.replace(".xml","")+"\n\n"
            graph_data=[]
            text_data=""
            for topic in category_list:
                cos_dis=np.mean([gensim.matutils.cossim(p1, p2) for p1, p2 in zip([centroids_dict[topic]], path)])
                text_data +=topic+":"+str(cos_dis)+"\n"
                graph_data.append(cos_dis)
            results+=text_data+"\n\n\n\n"
            for g in graph_data:
                print (round(g,2))
                
            draw_graph(list(centroids_dict.keys()),graph_data,root_folder+model_name+'/'+file,text_data,file)
output_file = open(root_folder+model_name+"/"+model_name+str(len(category_list))+".txt", 'w', encoding="utf8")
output_file.write(results)
output_file.close()
print(" Done ")

Testing results
Allegory.xml
[[0.040777338125449328, 0.19818742955168864, 0.022553908759205259, 0.051970215828235709, 0.0512840412000138, 0.3384683595989586, 0.15898905286120468, 0.015435762085985362, 0, 0.11463976746898566]]
0.95
0.88
0.78
0.92
0.85
0.54
0.79
0.84
0.87
0.57
Android.xml
[[0.040777338125449328, 0.19818742955168864, 0.022553908759205259, 0.051970215828235709, 0.0512840412000138, 0.3384683595989586, 0.15898905286120468, 0.015435762085985362, 0, 0.11463976746898566]]
0.22
0.26
0.42
0.17
0.19
0.13
0.3
0.12
0.15
0.66
Artificial Intelligence.xml
[[0.040777338125449328, 0.19818742955168864, 0.022553908759205259, 0.051970215828235709, 0.0512840412000138, 0.3384683595989586, 0.15898905286120468, 0.015435762085985362, 0, 0.11463976746898566]]
0.21
0.39
0.23
0.16
0.23
0.82
0.21
0.56
0.17
0.39
Cartoon Network.xml
[[0.040777338125449328, 0.19818742955168864, 0.022553908759205259, 0.051970215828235709, 0.0512840412000138, 0.3384683595989586, 0.15898905286120468, 0.015435762085985362,

In [42]:
# Test LDA- KMeans
model_name="LDA-KMeans"
if not os.path.exists(root_folder+model_name):
    os.mkdir(root_folder+model_name)
    
def get_part(testFile):
    test_doc = [tokens for tokens in iter_wiki(testFile)]
    part = [get_values(lda_model[id2word_wiki.doc2bow(tokens)]) for tokens in test_doc]
    return part

results=""
print("Testing results")
for path, subdirs, files in os.walk(test_folder):
    for file in files:
        if file.endswith('.xml'):
            doc_path = test_folder + file
            results+=file.replace(".xml","")+":  "
            part=get_part(doc_path)
            print(category_list[km.predict(part)[0]])
            results+=category_list[km.predict(part)[0]]
            results += "\n\n"
                
output_file = open(root_folder+model_name+"/"+model_name+str(len(category_list))+".txt", 'w', encoding="utf8")
output_file.write(results)
output_file.close()
print(" Done ")

Testing results
Geography
Nature
Arts
Nature
Nature
Arts
Arts
Health
Mathematics
History
History
Health
 Done 


In [41]:
#Test LDA _ KNN
model_name="LDA-KNN"
if not os.path.exists(root_folder+model_name):
    os.mkdir(root_folder+model_name)

def get_part(testFile):
    test_doc = [tokens for tokens in iter_wiki(testFile)]
    part = [get_values(lda_model[id2word_wiki.doc2bow(tokens)]) for tokens in test_doc]
    return part

results=""
print("Testing results")
for path, subdirs, files in os.walk(test_folder):
    for file in files:
        if file.endswith('.xml'):
            doc_path = test_folder + file
            results+=file.replace(".xml","")+":  "
            print(category_list[clf.predict(part)[0]])
            part=get_part(doc_path)
            results+=category_list[clf.predict(part)[0]]
            results += "\n\n"
                
output_file = open(root_folder+model_name+"/"+model_name+str(len(category_list))+".txt", 'w', encoding="utf8")
output_file.write(results)
output_file.close()
print(" Done ")
    

Testing results
Religion
Literature
Technology
Technology
Health
Technology
Mathematics
Mathematics
Health
Health
Health
Health
 Done 


In [34]:
#LDA Linear Classifier
model_name="LDA-Linear Classifier"

if not os.path.exists(root_folder+model_name):
    os.mkdir(root_folder+model_name)
    
def get_part(testFile):
    test_doc = [tokens for tokens in iter_wiki(testFile)]
    part = [get_values(lda_model[id2word_wiki.doc2bow(tokens)]) for tokens in test_doc]
    return part

print("Testing results")
for path, subdirs, files in os.walk(test_folder):
    for file in files:
        if file.endswith('.xml'):
            doc_path = test_folder + file
            results+=file.replace(".xml","")+":  "
            
            part=get_part(doc_path)
            #for i, clf in enumerate((svc, lin_svc, rbf_svc, poly_svc)):
            print(category_list[poly_svc.predict(part)[0]])
            #results +=category_list[clf.predict(part)[0]] + "\n\n"
                
output_file = open(root_folder+model_name+"/"+model_name+str(len(category_list))+".txt", 'w', encoding="utf8")
output_file.write(results)
output_file.close()
print(" Done ")

Testing results
Health
Health
Health
Health
Health
Health
Health
Health
Health
Health
Health
Health
 Done 


In [44]:

def calculate_centroid1(topic_docs):
    test_doc = [tokens for tokens in iter_wiki(topic_docs)]
    part = [lsi_model[id2word_wiki.doc2bow(tokens)] for tokens in test_doc]
    
    topic_dic={}
    
    for i in range(len(category_list)):
        topic_dic[i]=0
        
    for doc in part:
        for p in doc:
            topic_dic[p[0]] += p[1]
    
    centroid = [(x, topic_dic[x]/len(part)) for x in range(len(category_list))]
    return centroid

print("Calcuating centroid")

centroids_dict1={}
for path, subdirs, files in os.walk(root_folder):
    del subdirs[:]
    for file in files:
        if file.endswith('.xml'):
            doc_path = path + file
            centroid = calculate_centroid1(doc_path)
            centroids_dict1[file.replace(".xml","")]=centroid

print("Centroid calculation done")
model_name="LSI-Cosine"

if not os.path.exists(root_folder+model_name):
    os.mkdir(root_folder+model_name)
    
# Test 
def get_part1(testFile):
    test_doc = [tokens for tokens in iter_wiki(testFile)]
    part = [lsi_model[id2word_wiki.doc2bow(tokens)] for tokens in test_doc]
    return part

results=""
print("Testing results")
for path, subdirs, files in os.walk(test_folder):
    for file in files:
        if file.endswith('.xml'):
            doc_path = test_folder + file
            path=get_part1(doc_path)
            print(doc_path)
            results+=file.replace(".xml","")+"\n\n"
            graph_data=[]
            text_data=""
            graph_topic=[]
            for topic in category_list:
                
                cos_dis=np.mean([gensim.matutils.cossim(p1, p2) for p1, p2 in zip([centroids_dict[topic]], path)])
                text_data +=  topic+":"+str(cos_dis)+"\n"
                #if(np.mean([gensim.matutils.cossim(p1, p2) for p1, p2 in zip([centroids_dict11[topic]], path)])>0):
                graph_data.append(cos_dis)
                graph_topic.append(topic)
            for g in graph_data:
                print (round(g,2))
            results+=text_data+"\n\n\n\n"
            draw_graph(graph_topic,graph_data,root_folder+model_name+'/'+file,text_data,file)
output_file = open(root_folder+model_name+"/"+model_name+str(len(category_list))+".txt", 'w', encoding="utf8")
output_file.write(results)
output_file.close()
print(" Done ")

Calcuating centroid
Centroid calculation done
Testing results
./test/Allegory.xml
-0.12
0.01
-0.28
-0.14
-0.21
0.15
-0.3
0.11
-0.19
-0.29
./test/Android.xml
0.18
0.05
0.04
0.26
0.21
0.1
0.19
0.25
0.38
-0.25
./test/Artificial Intelligence.xml
-0.24
-0.15
-0.31
-0.25
-0.31
-0.03
-0.43
-0.06
-0.29
-0.35
./test/Cartoon Network.xml
-0.21
-0.26
-0.36
-0.16
-0.2
-0.19
-0.24
-0.07
-0.11
-0.52
./test/Computer Network.xml
-0.19
-0.26
-0.24
-0.16
-0.06
-0.01
-0.1
-0.03
-0.06
-0.44
./test/Computer Science.xml
-0.54
-0.47
-0.68
-0.53
-0.57
-0.25
-0.59
-0.38
-0.52
-0.7
./test/Dijkstra's Algorithm.xml
-0.14
-0.08
-0.21
-0.16
-0.14
0.17
-0.21
0.12
-0.13
-0.3
./test/Ebola.xml
-0.2
-0.17
-0.33
-0.15
-0.2
-0.09
-0.4
-0.11
-0.2
-0.52
./test/Fibonacci Heap.xml
-0.14
-0.08
-0.28
-0.17
-0.19
0.07
-0.3
0.05
-0.2
-0.4
./test/Google.xml
0.06
-0.02
-0.15
0.16
0.07
-0.05
-0.02
0.06
0.19
-0.43
./test/Larry Page.xml
0.04
0.04
-0.23
0.09
-0.03
-0.01
-0.1
0.09
0.06
-0.4
./test/USA.xml
-0.2
-0.18
-0.37
-0.11
-0.22
-0.

In [19]:
print("asd")
t=0
for path, subdirs, files in os.walk(test_folder):
    for file in files:
        if file.endswith('.xml'):
            #print(root_folder+file)
            
            csvReader = csv.reader(open(test_folder+file,'r'))
            #totalRecords = sum(1 for row in csv.reader(open(root_folder+file,'r',encoding="UTF-8")) )
            #t+=totalRecords
            print(file.replace(".xml",""))

asd
Allegory
Android
Artificial Intelligence
Cartoon Network
Computer Network
Computer Science
Dijkstra's Algorithm
Ebola
Fibonacci Heap
Google
Larry Page
USA
